In [15]:
# -*- coding: utf-8 -*-

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle

In [16]:
data=pd.read_excel("imdb_data.xlsx")
data

,Unnamed: 0,Movie Name,Year of Release,Watch Time,Genre,Movie Rating,Metascore of movie,Director,Cast,Votes,Description
0,0,Free Guy,2021,115 min,"\nAction, Comedy, Sci-Fi",7.6,62.0,Shawn Levy,"Ryan Reynolds,Jodie Comer,Taika Waititi","29,441",A bank teller discovers that he's actually an ...
1,1,The Suicide Squad,2021,132 min,"\nAction, Adventure, Comedy",7.4,72.0,James Gunn,"Margot Robbie,Idris Elba,John Cena","156,225","Supervillains Harley Quinn, Bloodsport, Peacem..."
2,2,Reminiscence,2021,116 min,"\nMystery, Romance, Sci-Fi",5.9,46.0,Lisa Joy,"Hugh Jackman,Rebecca Ferguson,Thandiwe Newton","11,879","Nick Bannister, a private investigator of the ..."
3,3,Beckett,2021,110 min,"\nAction, Crime, Drama",5.6,52.0,Ferdinando Cito Filomarino,"John David Washington,Boyd Holbrook,Vicky Krieps","14,267","Following a tragic car accident in Greece, Bec..."
4,4,Eternals,2021,NaN,"\nAction, Adventure, Drama",NaN,NaN,Chloé Zhao,"Richard Madden,Salma Hayek,Angelina Jolie",NaN,"The saga of the Eternals, a race of immortal b..."
...,...,...,...,...,...,...,...,...,...,...,...
995,995,The Good Shepherd,2006,167 min,"\nDrama, History, Thriller",6.7,61.0,Robert De Niro,"Matt Damon,Angelina Jolie,Robert De Niro","100,403",The tumultuous early history of the Central In...
996,996,Falling,I 2020,112 min,\nDrama,6.4,63.0,Viggo Mortensen,"Viggo Mortensen,Lance Henriksen,Terry Chen","3,975",John Peterson lives with his partner Eric and ...
997,997,2012,I 2009,158 min,"\nAction, Adventure, Sci-Fi",5.8,49.0,Roland Emmerich,"John Cusack,Thandiwe Newton,Chiwetel Ejiofor","362,537",A frustrated writer struggles to keep his fami...
998,998,Star Wars: Episode VI - Return of the Jedi,1983,131 min,"\nAction, Adventure, Fantasy",8.3,58.0,Richard Marquand,"Mark Hamill,Harrison Ford,Carrie Fisher","982,354",After a daring mission to rescue Han Solo from...


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          1000 non-null   int64  
 1   Movie Name          1000 non-null   object 
 2   Year of Release     1000 non-null   object 
 3   Watch Time          957 non-null    object 
 4   Genre               1000 non-null   object 
 5   Movie Rating        939 non-null    float64
 6   Metascore of movie  905 non-null    float64
 7   Director            1000 non-null   object 
 8   Cast                1000 non-null   object 
 9   Votes               939 non-null    object 
 10  Description         1000 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 86.1+ KB


In [18]:
# We rename some columns
data.rename(columns={'Unnamed: 0': 'movie_id'}, inplace=True)
data.rename(columns={'Movie Name': 'Title'}, inplace=True)

In [19]:
# Prepare a list of textual-data columns without null values
columns=['Title', 'Genre', 'Director', 'Cast', 'Description']



In [20]:
# Double check that those columns don't have nulls
data[columns].isnull().values.any() # no null values



np.False_

In [21]:
# Function to return the list of concatenated title, director, genre and description column
def get_important_features(data):
    important_features=[]
    for i in range (0, data.shape[0]):
        important_features.append(data['Title'][i]+ ' ' + data['Director'][i]+ ' ' +
                                  data['Genre'][i]+ ' ' + data['Description'][i])
    return important_features




In [22]:
# Saving the concatenated columns as a new feature
data['important_features'] = get_important_features(data)



In [23]:
# We vectorize this feature using TfidfVectorizer to calculate similarity
tfidf = TfidfVectorizer(stop_words='english')
#data['Description'] = data['Description'].fillna('')
tfidf_matrix = tfidf.fit_transform(data['important_features'])
tfidf_matrix.shape



(1000, 6723)

In [24]:
# Generating the cosine similrity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)



In [25]:
# We need to write a logic that takes a movie title as input and 
# returns the top 5 most similar movies based on cosine similarity.

 # For this, we can create a function that calculates the similarity
 # score of our input with other movies in the corpus, then sort the 
 # scores in descending order to get the top 5 movies with the 
 # highest similarity scores. Here index 0 is discarded in 
 # sim_scores variable so that the function does not return the 
 # same movie that’s been entered in the input.

indices = pd.Series(data.index, index=data['Title']).drop_duplicates()
#indices['Stillwater']
#sim_scores = list(enumerate(cosine_sim[indices['Stillwater']]))


In [26]:
indices['Stillwater']

np.int64(10)

In [30]:
sim_scores = list(enumerate(cosine_sim[indices['Stillwater']]))
print(sim_scores)
del sim_scores

[(0, np.float64(0.0)), (1, np.float64(0.04265320442226127)), (2, np.float64(0.0)), (3, np.float64(0.015225038821449184)), (4, np.float64(0.005222763042822635)), (5, np.float64(0.005365202130474507)), (6, np.float64(0.0)), (7, np.float64(0.0)), (8, np.float64(0.0)), (9, np.float64(0.055170534449780075)), (10, np.float64(1.0000000000000002)), (11, np.float64(0.0)), (12, np.float64(0.0046193460431244214)), (13, np.float64(0.009891443434499795)), (14, np.float64(0.01630460221665394)), (15, np.float64(0.0)), (16, np.float64(0.07937999103949504)), (17, np.float64(0.004465363750495506)), (18, np.float64(0.0)), (19, np.float64(0.04631130246556378)), (20, np.float64(0.0)), (21, np.float64(0.0)), (22, np.float64(0.015511792931630446)), (23, np.float64(0.011764106668597223)), (24, np.float64(0.004428350658830342)), (25, np.float64(0.021278008829661372)), (26, np.float64(0.005925785908722934)), (27, np.float64(0.0)), (28, np.float64(0.013091984225537806)), (29, np.float64(0.013926102406456201)), (

In [31]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 5 most similar movies
    movies=data['Title'].iloc[movie_indices]
    id=data['movie_id'].iloc[movie_indices]
    dict={"Movies":movies,"id":id}
    final_df=pd.DataFrame(dict)
    final_df.reset_index(drop=True,inplace=True)
    return final_df



In [32]:
# Test our recommendation engine
get_recommendations('Spider-Man: Far from Home')


,Movies,id
0,Spider-Man: No Way Home,39
1,Spider-Man: Homecoming,220
2,Spider-Man: Into the Spider-Verse,341
3,Spider-Man 4: Fan Film,331
4,Spider-Man,383


In [33]:
#Stillwater
get_recommendations('Stillwater')



,Movies,id
0,Taken,868
1,Les Misérables,686
2,Spotlight,425
3,Road to Perdition,867
4,Minority Report,784


In [34]:
get_recommendations('Goodfellas')

,Movies,id
0,The Wolf of Wall Street,64
1,The Irishman,277
2,Pulp Fiction,91
3,Looper,748
4,Notting Hill,848


In [35]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            1000 non-null   int64  
 1   Title               1000 non-null   object 
 2   Year of Release     1000 non-null   object 
 3   Watch Time          957 non-null    object 
 4   Genre               1000 non-null   object 
 5   Movie Rating        939 non-null    float64
 6   Metascore of movie  905 non-null    float64
 7   Director            1000 non-null   object 
 8   Cast                1000 non-null   object 
 9   Votes               939 non-null    object 
 10  Description         1000 non-null   object 
 11  important_features  1000 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 93.9+ KB


In [36]:
new = data.drop(columns=['Year of Release','Watch Time','Genre','Movie Rating','Metascore of movie','Director','Cast','Votes','Description'])



In [37]:
new

,movie_id,Title,important_features
0,0,Free Guy,"Free Guy Shawn Levy \nAction, Comedy, Sci-Fi ..."
1,1,The Suicide Squad,"The Suicide Squad James Gunn \nAction, Adventu..."
2,2,Reminiscence,"Reminiscence Lisa Joy \nMystery, Romance, Sci-..."
3,3,Beckett,"Beckett Ferdinando Cito Filomarino \nAction, C..."
4,4,Eternals,"Eternals Chloé Zhao \nAction, Adventure, Drama..."
...,...,...,...
995,995,The Good Shepherd,"The Good Shepherd Robert De Niro \nDrama, Hist..."
996,996,Falling,Falling Viggo Mortensen \nDrama Jo...
997,997,2012,"2012 Roland Emmerich \nAction, Adventure, Sci-..."
998,998,Star Wars: Episode VI - Return of the Jedi,Star Wars: Episode VI - Return of the Jedi Ric...


In [38]:
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(cosine_sim,open('similarity.pkl','wb'))